In [ ]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
import pyarrow as pa
import pyarrow.parquet as pq
import gcsfs
import getpass
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import geopandas as gpd
import sgis as sg
import dapla as dp
import datetime
from dapla.auth import AuthClient
from dapla import FileClient
from sklearn.metrics import mean_absolute_error, r2_score

# from graphviz import Digrap


import sys

sys.path.append("../functions")

import knn
import networks

# import plan
import kommune
import input_data

fs = FileClient.get_gcs_file_system()
import numpy as np


import warnings

warnings.filterwarnings("ignore")


# aar = input("Årgang: ")
# enhet = input("orgnr_foretak #: ")

aar = "2021"
enhet = "879263662"

# enhet_df, merged_gdf, reg_type_02_df, adjusted_result_df, naermeste_naboer = knn.knn(
#     aar, enhet
# )

---------------------------------------------------------------------------------------------------------------------------------



<div style="background-color: #F0F0F0; padding: 10px; font-size: 32px; font-weight: bold; text-align: center; border-radius: 10px;">
  <span style="color: green;">Maskinlæringsløsninger for produksjonen av all statistikk i S423</span>
</div>



---------------------------------------------------------------------------------------------------------------------------------

# Planen

### Kortsiktig plan (innen 1 år eller tidligere):

- Utvikle en algoritme basert på maskinlæring som automatisk kan oppdatere bedriftslisten for varehandel - inkludert foretak som ikke leverer noen data.
- Fullstendig automatisere nøkkelindikatorer for næring 45.
- Erstatte nærmeste nabo-funksjonene i Nøkkelindikatorproduksjonssystemet (Viggens program) med mer moderne/nøyaktige maskinlæringsverktøy.
- Samarbeide med VoF og S422 for å samle nyttige data for å hjelpe med å trene modeller. (Har allerede hatt noen samtaler med folk fra disse avdelingene). Vi har nå muligheten til å produsere statistikk av mye høyere kvalitet basert på det de kan tilby oss.
- Rulle ut løsningen til andre grupper i S423.

#### Her er hvordan den tidlige algoritmen vil fungere.
##### Gir gode resultater tidlig i prosessen - med enkle forbedringer som ennå kan gjøres. Dette bør gjøre det mulig å slippe å erstatte Madli.

In [ ]:
from IPython.display import Image

# Provide the path to your image file
image_path = "flow chart.png"

# Display the image
Image(filename=image_path)

### Langsiktig plan (1-3 år)

- Fullstendig overskrive Viggens program. Start helt fra bunnen av og bruk moderne løsninger. Spesielt bør vi bruke data fra andre avsnitt der det er mulig og bruke ML for å estimere variabler der vi tidligere bare brukte hardkodede funksjoner. Nevrale nettverk kan kanskje finne løsninger per næring.
- Eliminere behovet for bedriftsfordeling av omsetning og driftskostnader. Vi kan fjerne det fra OKI-skjemaene.
- Produsere mer/bedre statistikk. Vi vil ha mer tid, derfor kan vi publisere nye og mer interessante statistikker.



- Etter min mening, til tross for at vi har startet et editeringsløsning-team, har vi en veldig reell mulighet til å ikke trenge et editeringsprodukt i S423. Vi kan kontrollere og rydde våre data ved hjelp av mer moderne løsninger.

# Koden så langt - KNN modell

---------------------------------------------------------------------------------------------------------------------------------


### Har startet med en KNN-modell. Gode resultater så langt. Vil også legge til flere funksjoner i treningsdatasettet - skal teste også andre modeller f.eks:

- Decision Tree
- Random Forest 
- Neural Networks

---------------------------------------------------------------------------------------------------------------------------------

### Kjør programmet
##### Eksempel:
##### Næring : 45.200

- 879263662

- K-nærmeste naboer blir funnet ved hjelp av en maskinlæringsalgoritme som forsøker å minimere feil ved å bruke geografisk avstand, omsetning, sysselsetting og oms per syss som variabler for å trene modellen.
- Vi kan legge til og fjerne disse treningsvariablene som vi vil. Vi kan bruke flere verktøy for å teste hvilke variabler som er mest nyttige - også kan vi bruke nevrale nettverk for å finne de riktige variablene automatisk.
- S422 har noen variabler som jeg tror vil være til stor hjelp.

# Kartlegging av næringen

In [ ]:
sg.explore(merged_gdf, "omsetning")

# Kartlegging av foretaket

In [ ]:
sg.explore(enhet_df, "omsetning")

# Å finne nærmeste naboer

In [ ]:
sg.explore(naermeste_naboer, "orgnr_foretak")

# Evaluering av resultater for reg_type 2 virksomheter

In [ ]:
# Calculate Mean Absolute Error (MAE) using the adjusted predictions
mae_new_predicted = mean_absolute_error(
    reg_type_02_df["actual_omsetning"], reg_type_02_df["new_predicted"]
)
r2_new_predicted = r2_score(
    reg_type_02_df["actual_omsetning"], reg_type_02_df["new_predicted"]
)
print(
    f"Mean Absolute Error (MAE) / Gjennomsnittlig absolutt feil for reg_type 2: {mae_new_predicted}"
)
print(f"R-kvadrert (R2) for reg_type 2: {r2_new_predicted}")

# # Evaluering av resultater for reg_type 2 virksomheter etter optimisere funksjon

In [ ]:
mae_new_predicted = mean_absolute_error(
    adjusted_result_df["actual_omsetning"], adjusted_result_df["new_predicted"]
)
r2_new_predicted = r2_score(
    adjusted_result_df["actual_omsetning"], adjusted_result_df["new_predicted"]
)
print(
    f"Mean Absolute Error (MAE)/ Gjennomsnittlig absolutt feil for hele næringen: {mae_new_predicted}"
)
print(f"R-squared (R2) for hele næringen: {r2_new_predicted}")

# Resultater 

In [ ]:
reg_type_02_df

--------------------------------------------------------------------------------------------------------------------------------

<div style="background-color: #F0F0F0; padding: 10px; font-size: 32px; font-weight: bold; text-align: center; border-radius: 10px;">
  <span style="color: green;">Veien videre</span>
</div>

--------------------------------------------------------------------------------------------------------------------------------

# Samarbeid med S422 og S424 og data fra avdeling 300

#### Stort potensial her: 

- Jeg har snakket med S422, og de har transaksjonsdata på virksomhetsnivå. Dette ville ikke bare eliminere behovet for å be om bedriftsfordeling i OKI-skjemaet (først for varehandel OG tjenesteyting, men for alle sammen litt senere) , men det ville også være svært kraftige data å trene maskinlæringsmodeller på , noe som forbedrer deres nøyaktighet. De har også investeringsdata som ville være svært nyttige for vår avdeling.

- Selv om vi ikke kan eliminere behovet for bedriftsfordeling i OKI-skjemaet for alle gruppene ennå, kan nok transaksjonsdataene vi har for industri/bygg osv. være svært kraftige for å trene opp en automatisk rettingmodell for disse næringer.

- Jeg har fått tilgang til noen av datasettene hos VoF, som gjør det mulig for oss å kartlegge og koble våre statistikker med hvor virksomhetene befinner seg, ikke bare geografisk, men også i forhold til ting som reisetid, antall kjøretøy som passerer virksomheten osv.

- Jeg har også brukt data som jeg har hentet fra avdeling 300 for å lage statistikk over befolkningstetthet og inntektstetthet for å kartlegge til våre virksomheter - jeg tror dette har potensial til å hjelpe oss med å trene svært nyttige modeller. Dette kan ses nedenfor:

### Legg til flere treningsvariabler

In [ ]:
kommuner = kommune.kommune()

In [ ]:
m = sg.ThematicMap(enhet_df, kommuner, column="befolkning", size=15)
m.title = "Befolkning i kommunene"
m.plot()

In [ ]:
m = sg.ThematicMap(enhet_df, kommuner, column="inntekt", size=15)
m.title = "Inntekt i kommunene"
m.plot()

# Nettverksanalyse

In [ ]:
frequencies = networks.networks(aar)

In [ ]:
filtered_df = enhet_df[enhet_df["kommune"].str.startswith("03")]

In [ ]:
m = sg.ThematicMap(sg.buff(frequencies, 18), filtered_df, column="frequency", size=15)
m.black = True
m.cmap = "plasma"  # fargepaletten
m.title = "Antall ganger hver vei ble brukt"
m.legend.title = "Antall"
m.plot()

# Trender over flere tidsperioder

Kan spørre Statbank API

In [ ]:
# Provide the path to your image file
image_path = "trend.png"

# Display the image
Image(filename=image_path)

--------------------------------------------------------------------------------------------------------------------------------

<div style="background-color: #F0F0F0; padding: 10px; font-size: 32px; font-weight: bold; text-align: center; border-radius: 10px;">
  <span style="color: green;">Modellvalg</span>
</div>

--------------------------------------------------------------------------------------------------------------------------------

- KNN (K- nærmeste naboer)
- Beslutningstær (Decision Trees)
- Tilfeldige skoger (Random Forests)
- Nevrale nettverk (Neural Networks)


# KNN

KNN, eller k-nærmeste naboer, er en type maskinlæringsmodell som brukes for klassifisering og regresjon. I KNN-algoritmen, for å gjøre en prediksjon for en ny datapunkt, finner algoritmen de 'k' nærmeste datapunktene i treningssettet og gjør en prediksjon basert på disse.

Fordeler med KNN:
1. Enkel å forstå og implementere.
2. Modellen tilpasser seg automatisk i takt med at mer data blir tilgjengelig.
3. Effektiv for datasett med et lite antall dimensjoner.

Ulemper med KNN:
1. Kan være treg for store datasett, da den krever beregning av avstanden til hvert punkt i treningssettet.
2. Sensitiv for skalerte data og irrelevant funksjoner, noe som kan påvirke nøyaktigheten.
3. Krever en god definisjon av hva som utgjør "nærhet", noe som kan være utfordrende i noen domener.

# Beslutningstær (Decision Trees)

Beslutningstrær er en type maskinlæringsmodell som brukes for både klassifisering og regresjon. I beslutningstrær lages en modell som forutsier verdien av en målvariabel basert på enkle beslutningsregler utledet fra dataegenskapene.

Fordeler med beslutningstrær:
1. Enkle å forstå og visualisere, noe som gjør dem tilgjengelige for brukere med ikke-teknisk bakgrunn.
2. Krever lite dataforberedelse sammenlignet med andre algoritmer.
3. Kan håndtere både numeriske og kategoriske data.

Ulemper med beslutningstrær:
1. Kan lett overtilpasse seg dataene, noe som fører til dårlig generalisering på nye data.
2. Er følsomme for små endringer i dataene, noe som kan føre til store endringer i strukturen på treet.
3. Kan være partiske mot de funksjonene som har flere nivåer.

# Tilfeldige skoger (Random Forests)

Tilfeldige skoger er en ensemble-læringsmetode for klassifisering, regresjon og andre oppgaver. Den opererer ved å konstruere en mengde beslutningstrær ved treningstidspunktet og utleder den endelige beslutningen basert på flertallet av trærne eller gjennomsnittet av deres prediksjoner.

Fordeler med tilfeldige skoger:
1. Reduserer risikoen for overtilpasning som er vanlig i enkeltstående beslutningstrær.
2. Effektiv på store datasett og kan håndtere tusenvis av inngangsvariabler uten variabeldeletelse.
3. Gir en indikasjon på hvilke funksjoner som er viktigst i klassifiseringen.

Ulemper med tilfeldige skoger:
1. Modellene kan bli ganske store og krever mye ressurser for både trening og prediksjon.
2. Mindre intuitiv og vanskeligere å visualisere sammenlignet med enkeltstående beslutningstrær.
3. Prediksjonene fra tilfeldige skoger er mindre tolkbare enn de fra beslutningstrær.

# Nevrale nettverk (Neural Network)

Nevrale nettverk er en klasse av maskinlæringsalgoritmer som er inspirert av strukturen og funksjonene til menneskelige hjerner. De består av lag med knutepunkter (nevroner) som simulerer hvordan hjernen behandler og reagerer på inngående informasjon.

Fordeler med nevrale nettverk:
1. Evne til å lære og modellere ikke-lineære og komplekse relasjoner.
2. Etter tilstrekkelig trening, er de ekstremt nøyaktige i prediksjoner.
3. Fleksible til å tilpasse seg ulike typer data og kan håndtere både klassifisering og regresjonsoppgaver.

Ulemper med nevrale nettverk:
1. Krever store mengder data for å trene effektivt.
2. Tidskrevende og ressursintensiv trening, spesielt for større nettverk.
3. Modellens beslutninger er ofte ikke gjennomsiktige (kjent som "black box" problemet), noe som gjør det vanskelig å forstå hvorfor nettverket har tatt en bestemt beslutning.

# Modell Stacking

Vi kan 'stack' modeller (bruke felere modeller samtidig). I AO programmet finner du et eksempel nederst. KNN og simple linear regresseion brukes (pre - optimisering) - og Mean Absolute Error gikk ned fra 79725 til  41684

# Production RUN

In [ ]:
year = 2021
fjor = year - 1
limit = 0.6
good_df, onlygoodoms, bad_df, merged_df = input_data.split_input_data(year, limit)